<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [ ]:
!gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
!unzip archive.zip

Downloading...
From: https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph
To: /content/archive.zip
84.9MB [00:00, 104MB/s] 
Archive:  archive.zip
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Импортируем нужные библиотеки:

In [ ]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [ ]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [ ]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [ ]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


In [ ]:
examples = data["text"]

Как вилим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [ ]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [ ]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ claire_nelson i ' m on the north devon coast the next few weeks will be down in devon again in may sometime i hope though !


In [ ]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

north devon coast next weeks will down devon again sometime hope though


Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [ ]:
word2vec = api.load("word2vec-google-news-300")

In [ ]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

In [ ]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False, False, False, False, False, False]


In [ ]:
# txt = dev_data['text'][0]
# print(txt)
# tkns = dev.get_tokens_(txt)
# print(tkns)
# embs = dev.get_embeddings_(tkns)
# print(embs)

# txt = dev_data['text'][5]
# print(txt)
# tkns = dev.get_tokens_(txt)
# print(tkns)
# embs = dev.get_embeddings_(tkns)
# print(embs)

embeddings_1 = [1, 2, 3, 4]
embeddings_2 = [4, 5, 6, 7]
tok = embeddings_1, embeddings_2
# em = np.array([]) # np.empty((0, 3), int)
# em.shape
# for i in tok:
#     em = np.append(em, np.array(i))
# #em = np.array([embeddings_1, embeddings_2])
# print(cb)
# t = np.mean(cb, axis=0)
# print(t)

import numpy as np    
arr = np.empty((0,4), int)
a = np.zeros(4)
print("Empty array:")
print(arr)
for i in tok:
    a += i
    arr = np.append(arr, np.array([i]), axis=0)
# arr = np.append(arr, np.array([[10,20,30]]), axis=0)
# arr = np.append(arr, np.array([[40,50,60]]), axis=0)
a = a / 2
print("Arr:", arr)
print(np.mean(a, axis=0))
print(a)

Empty array:
[]
Arr: [[1 2 3 4]
 [4 5 6 7]]
4.0
[2.5 3.5 4.5 5.5]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [ ]:
from torch.utils.data import Dataset, random_split

class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        pre_token = tokenizer.tokenize(text.lower())
        filt_token = [w for w in pre_token if all(c not in string.punctuation for c in w) and len(w) > 3]

        tokens = []
        for txt in filt_token:
            if txt in word2vec:
                tokens.append(txt)
        return tokens
        # Получи все токены из текста и профильтруй их

    def get_embeddings_(self, tokens):
        embeddings = []
        avg_em = np.zeros(self.word2vec.vector_size)
        count = 0
        for emb in tokens:
            #embeddings = [((word2vec.get_vector(w) - self.mean) / self.std) for w in tokens]
            #embeddings.append(word2vec.get_vector(emb)) # / len(tokens) # Получи эмбеддинги слов и усредни их
            norm_emb_line = (word2vec.get_vector(emb) - self.mean) / self.std
            #embeddings.append(norm_emb_line)
            #np.append(embeddings, norm_emb_line) 
            avg_em += norm_emb_line #word2vec.get_vector(emb)
            count += 1
        if count > 0: #len(embeddings) > 0: #count > 0:
            avg_em = avg_em / count
            embeddings = [avg_em]
            #embeddings = np.average(embeddings, axis=0)

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            #print(embeddings.shape)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

In [ ]:
# embeddings = np.zeros((1, 300))
# print(np.sum(embeddings, axis=0))
# embeddings = embeddings.reshape(-1, 1)
#print(embeddings)
tok = dev.get_tokens_("hello wordl road")
em = dev.get_embeddings_(tok)
print(em)

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(examples["features"])# Обучи PCA на эмбеддингах слов

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1002', ...)

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [ ]:
from torch.utils.data import DataLoader
batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

Определим функции для тренировки и теста модели:

In [ ]:
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score

def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        # print(batch)
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        y_pred = model(features)

        # print(y_pred)
        # print(targets)

        # Получи предсказания модели
        loss = criterion(y_pred, targets) # Посчитай лосс
        # print(loss)
        # Обнови параметры модели
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
        # break
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            # Получи предсказания модели
            y_pred = model(features)
            loss = criterion(y_pred, targets) # Посчитай лосс
            targ = targets.detach().cpu().clone()
            #pred = torch.round(pred)
            pred = torch.argmax(y_pred, dim=1)
            pred = pred.detach().cpu().clone()
            targ = targ.numpy()
            pred = pred.numpy()
            #print("targ: ", targ)
            #print("pred: ", pred)
            acc = accuracy_score(targ, pred) # torch.sum(torch.argmax(y_pred, dim=1) == targets).float() #accuracy_score(targets.cpu(), (torch.round(y_pred)).cpu())# Посчитай точность модели

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


In [ ]:
import torch.nn as nn

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.enc = nn.Sequential(
            nn.Linear(300, 100),
            nn.ReLU(),
            nn.Linear(100, 10),
            nn.ReLU(),
            nn.Linear(10, 2),
            nn.Sigmoid()
            #nn.Softmax()
            # nn.Sigmoid()
        )
        # self.u = nn.Embedding(voc_size, emb_dim)
        #self.v = nn.Linear(emb_dim, voc_size, bias=False)
        
    def forward(self, x):
        output = self.enc(x)
        return output #self.v(self.u(x))

In [ ]:
import torch.nn as nn
from torch.optim import Adam
#from sklearn.svm import LinearSVC

# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-3
num_epochs = 5

model_ = Model() #nn.Linear(vector_size, num_classes) #Model(len(dev), vector_size) # Твоя модель
#model_ = nn.Linear(300, 2)
model = model_.cuda()
criterion = nn.CrossEntropyLoss() # Твой лосс
optimizer = torch.optim.Adam(model.parameters(), lr) # Твой оптимайзер

RuntimeError: ignored

Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.5664250733852386, 'Test Acc': 0.72802734375}



{'Test Loss': 0.5596969847679139, 'Test Acc': 0.735796875}



{'Test Loss': 0.5547546210289002, 'Test Acc': 0.74097265625}



{'Test Loss': 0.5515460221767425, 'Test Acc': 0.74489453125}



{'Test Loss': 0.5501173436641693, 'Test Acc': 0.74630078125}



{'Test Loss': 0.5487787935733796, 'Test Acc': 0.7479453125}



{'Test Loss': 0.5471611196994781, 'Test Acc': 0.75009375}



{'Test Loss': 0.5468051104545594, 'Test Acc': 0.749921875}



{'Test Loss': 0.5463993992805481, 'Test Acc': 0.7502578125}



{'Test Loss': 0.5445066130161286, 'Test Acc': 0.75308203125}


In [ ]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

NameError: ignored

## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого 
слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

In [ ]:
t = ['dadwa', 'dadwdx', 'dawdsd', 'dwdw']
len(t)
#t[len(t) - 1]

4

In [ ]:
def check_word2vec(text):
    if text in word2vec: 
        return True
    else:
        return False

unknown_word_emb_map = {}
unknown_word_cnt_map = {}
GET_ONLY_MAPS = True

def get_maps(data):
    global unknown_word_emb_map
    global unknown_word_cnt_map

    for text in data["text"]:
        pre_token = tokenizer.tokenize(text.lower())
        tokens = [w for w in pre_token if all(c not in string.punctuation for c in w) and len(w) > 3] 

        for t in range(len(tokens)):
              if tokens[t] not in word2vec:
                add_cnt = 0
                left_emb = np.zeros(word2vec.vector_size)
                rght_emb = np.zeros(word2vec.vector_size)
                if t > 0 and check_word2vec(tokens[t - 1]):
                  left_emb += word2vec.get_vector(tokens[t - 1])
                  add_cnt += 1
                if t < len(tokens) - 1 and check_word2vec(tokens[t + 1]):
                  rght_emb += word2vec.get_vector(tokens[t + 1])
                  add_cnt += 1
                if tokens[t] in unknown_word_emb_map.keys():
                  unknown_word_emb_map[tokens[t]] += (left_emb + rght_emb)
                  unknown_word_cnt_map[tokens[t]] += add_cnt
                else:
                  unknown_word_emb_map[tokens[t]] = (left_emb + rght_emb)
                  unknown_word_cnt_map[tokens[t]] = add_cnt

class TwitterDataset_for_unknown_words(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        
        embeddings = self.get_embeddings_(tokens)
        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        pre_token = tokenizer.tokenize(text.lower())
        tokens = [w for w in pre_token if all(c not in string.punctuation for c in w) and len(w) > 3]
        filt_token = []

        return tokens


    def get_embeddings_(self, tokens):
        embeddings = []
        avg_em = np.zeros(self.word2vec.vector_size)
        count = 0
        for emb in tokens:
            if emb in self.word2vec:
                # norm_emb_line = (word2vec.get_vector(emb) - self.mean) / self.std
                embeddings.append(self.word2vec.get_vector(emb))
                #avg_em += self.word2vec.get_vector(emb)
                #count += 1
            else:
                if emb in unknown_word_emb_map:
                    embeddings.append(unknown_word_emb_map[emb])
                    # avg_em += (unknown_word_emb_map[emb]) # / unknown_word_cnt_map[emb])
                    # count += 1
        if count > 0:
            avg_em = avg_em / count
            embeddings = [avg_em]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            #print(embeddings.shape)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
get_maps(dev_data)

In [ ]:
#print(len(unknown_word_emb_map))
unknown_word_cnt_map

In [ ]:
dev_unknown_words = TwitterDataset_for_unknown_words(dev_data, "text", "emotion", word2vec)

In [ ]:
tok = dev_unknown_words.get_tokens_("hello wordl road")
em = dev_unknown_words.get_embeddings_(tok)
print(em)

In [ ]:
batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev_unknown_words) * 0.8)

train, valid = random_split(dev_unknown_words, [train_size, len(dev_unknown_words) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
len(dev_unknown_words)

1280000

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)



{'Test Loss': 0.6134899055957794, 'Test Acc': 0.72584375}



{'Test Loss': 0.6134345438480377, 'Test Acc': 0.7260078125}



{'Test Loss': 0.6134744665622711, 'Test Acc': 0.72606640625}



{'Test Loss': 0.6135210123062134, 'Test Acc': 0.725734375}



{'Test Loss': 0.613519535779953, 'Test Acc': 0.72566015625}


In [ ]:
test_loader = DataLoader(
    TwitterDataset_for_unknown_words(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)



{'Test Loss': 0.6137879593684651, 'Test Acc': 0.7250212160543131}


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
train_size_vec = math.ceil(len(data["text"]) * 0.8)
train_vec, valid_vec = random_split(data["text"], [train_size_vec, len(data["text"]) - train_size_vec])

In [ ]:
data["text"][1]

"is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english',min_df=0.003, max_df=0.95, tokenizer=tokenizer.tokenize, max_features=300) #, tokenizer=tokenizer.tokenize)
vec = vectorizer.fit_transform(train_vec)

In [ ]:
vectorizer.get_feature_names()
#vec.get

In [ ]:
dance = vec.todense()

In [ ]:
txt = data["text"][1]
print(txt)
d = vectorizer.transform([txt])
print(d)
d = d.todense()
d = np.sum(d, axis=0)
d

is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!
  (0, 252)	0.4615062786466137
  (0, 235)	0.3584982032102778
  (0, 217)	0.5965021839958103
  (0, 15)	0.35146978758942715
  (0, 13)	0.21666830136676538
  (0, 6)	0.24208727169517794
  (0, 0)	0.27128163600003136


matrix([[0.27128164, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.24208727, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.2166683 , 0.        ,
         0.35146979, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
svdT = TruncatedSVD(n_components=300)
svdFit = svdT.fit_transform(vec)

In [ ]:
vectorizer.get_feature_names()

In [ ]:
v = vectorizer.vocabulary_.get('too')
v

246

In [ ]:
vec[v].toarray()

TypeError: ignored

In [ ]:
class TwitterDatasetVectorizer(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        
        embeddings = self.get_embeddings_(tokens)
        vec = vectorizer.transform([text])

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        pre_token = tokenizer.tokenize(text.lower())
        tokens = [w for w in pre_token if all(c not in string.punctuation for c in w) and len(w) > 3]

        return tokens


    def get_embeddings_(self, tokens):
        embeddings = []

        for emb in tokens:
            vec = vectorizer.transform([emb])
            if emb in word2vec:
                e = word2vec.get_vector(emb) + vec
                embeddings.append(e)
            else:
                embeddings.append(vec)

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            #print(embeddings.shape)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev_vec = TwitterDatasetVectorizer(dev_data, "text", "emotion", word2vec)

In [ ]:
batch_size = 1024
num_workers = 4

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev_vec) * 0.8)

train_vec, valid_vec = random_split(dev_vec, [train_size, len(dev_vec) - train_size])

train_loader_vec = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader_vec = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader_vec, e, device)
    log = testing(model, criterion, valid_loader_vec, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]


{'Test Loss': 0.5540808670520783, 'Test Acc': 0.74214453125}



{'Test Loss': 0.5476516473293305, 'Test Acc': 0.74908984375}



{'Test Loss': 0.5454866588115692, 'Test Acc': 0.75103125}



{'Test Loss': 0.5439522910118103, 'Test Acc': 0.7525078125}



{'Test Loss': 0.5432597825527191, 'Test Acc': 0.75338671875}
